# `Project Overview`

This project builds a Next Word Prediction model using an LSTM neural network trained on Shakespeare’s Hamlet text. The model learns language patterns, word relationships, and context by processing sequences of words and predicting the most likely next word. The dataset is taken from the NLTK Gutenberg corpus and saved locally for training. An embedding layer converts words into vector representations, stacked LSTM layers learn long-term dependencies in the text, and a softmax output layer predicts the next word from the vocabulary. This project demonstrates how deep learning can be used for language modeling and text generation.


# `Importing Libraries`

In [1]:
import tensorflow as tf
print("TF version:", tf.__version__)
print("GPUs detected:", tf.config.list_physical_devices('GPU'))


gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

TF version: 2.10.0
GPUs detected: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
# Data Collection

import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg

data = gutenberg.raw('shakespeare-hamlet.txt')

with open('data_hamlet.txt','w') as file:
    file.write(data)

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\saina\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [3]:
import numpy as np
import pandas as pd
import pickle

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, GRU

from sklearn.model_selection import train_test_split

np.set_printoptions(linewidth=120)

In [4]:
with open('data_hamlet.txt','r') as file:
    data = file.read().lower()   # This is String

In [5]:
# Tokenize the text (Giving Index to Each Word)
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

with open('Tokenizer_File.pickle','wb') as handle:
    pickle.dump(tokenizer, handle)

In [6]:
# type(tokenizer.word_index)  # Word -> Index (Key Value Pairs)
total_n_words = len(tokenizer.word_index) + 1
total_n_words

4818

In [7]:
# Create n-gram input sequences
sequences = []

for sentence in data.split("\n"):
    tokens = tokenizer.texts_to_sequences([sentence])[0]

    for i in range(1, len(tokens)):
        sequence = tokens[:i + 1]
        sequences.append(sequence)
        
sequences[:20]

# Here at Input / Output Division 
# The Last Item Will Be removed and Assigned in y

# [1]                      predict 687
# [1, 687]                 predict 4
# [1, 687, 4]              predict 45
# [1, 687, 4, 45]          predict 41
# [1, 687, 4, 45, 41]      predict 1886
# ...

[[1, 687],
 [1, 687, 4],
 [1, 687, 4, 45],
 [1, 687, 4, 45, 41],
 [1, 687, 4, 45, 41, 1886],
 [1, 687, 4, 45, 41, 1886, 1887],
 [1, 687, 4, 45, 41, 1886, 1887, 1888],
 [1180, 1889],
 [1180, 1889, 1890],
 [1180, 1889, 1890, 1891],
 [57, 407],
 [57, 407, 2],
 [57, 407, 2, 1181],
 [57, 407, 2, 1181, 177],
 [57, 407, 2, 1181, 177, 1892],
 [407, 1182],
 [407, 1182, 63],
 [408, 162],
 [408, 162, 377],
 [408, 162, 377, 21]]

In [8]:
# Pad Sequences
max_len_sequences =max([len(x) for x in sequences])

sequences = np.array(pad_sequences(sequences, maxlen=max_len_sequences, padding='pre'))
sequences[:20]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    1,  687],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    1,  687,    4],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    1,  687,    4,   45],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    1,  687,    4,   45,   41],
       [   0,    0,    0,    0,    0,    0,    0,    0,    1,  687,    4,   45,   41, 1886],
       [   0,    0,    0,    0,    0,    0,    0,    1,  687,    4,   45,   41, 1886, 1887],
       [   0,    0,    0,    0,    0,    0,    1,  687,    4,   45,   41, 1886, 1887, 1888],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1180, 1889],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1180, 1889, 1890],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1180, 1889, 1890, 1891],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,   

In [9]:
X = sequences[:,:-1]
y = sequences[:,-1]

X.shape, y.shape, X, y

((25732, 13),
 (25732,),
 array([[   0,    0,    0, ...,    0,    0,    1],
        [   0,    0,    0, ...,    0,    1,  687],
        [   0,    0,    0, ...,    1,  687,    4],
        ...,
        [   0,    0,    0, ...,  687,    4,   45],
        [   0,    0,    0, ...,    4,   45, 1047],
        [   0,    0,    0, ...,   45, 1047,    4]]),
 array([ 687,    4,   45, ..., 1047,    4,  193]))

In [10]:
y = tf.keras.utils.to_categorical(y, num_classes = total_n_words)
y.shape, y

((25732, 4818),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32))

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# `Training Model`

In [12]:
model = Sequential([
    Embedding(input_dim = total_n_words, output_dim = 100, input_length = max_len_sequences - 1),
    LSTM(150, return_sequences = True),
    Dropout(0.2),
    LSTM(100),
    Dense(units = total_n_words, activation = "softmax")
])

model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 13, 100)           481800    
                                                                 
 lstm (LSTM)                 (None, 13, 150)           150600    
                                                                 
 dropout (Dropout)           (None, 13, 150)           0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               100400    
                                                                 
 dense (Dense)               (None, 4818)              486618    
                                                                 
Total params: 1,219,418
Trainable params: 1,219,418
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Training the Model
LSTM_model_history = model.fit(X_train, y_train,
                               epochs = 150, 
                               validation_data=(X_test,y_test),
                               verbose=1)

Epoch 1/150
644/644 [==============================] - 25s 28ms/step - loss: 6.9087 - accuracy: 0.0334 - val_loss: 6.7223 - val_accuracy: 0.0334
Epoch 2/150
644/644 [==============================] - 17s 26ms/step - loss: 6.4791 - accuracy: 0.0394 - val_loss: 6.8001 - val_accuracy: 0.0394
Epoch 3/150
644/644 [==============================] - 17s 26ms/step - loss: 6.3361 - accuracy: 0.0471 - val_loss: 6.8314 - val_accuracy: 0.0480
Epoch 4/150
644/644 [==============================] - 17s 26ms/step - loss: 6.1798 - accuracy: 0.0522 - val_loss: 6.8644 - val_accuracy: 0.0521
Epoch 5/150
644/644 [==============================] - 17s 26ms/step - loss: 6.0331 - accuracy: 0.0570 - val_loss: 6.8938 - val_accuracy: 0.0573
Epoch 6/150
644/644 [==============================] - 17s 26ms/step - loss: 5.8727 - accuracy: 0.0670 - val_loss: 6.9500 - val_accuracy: 0.0626
Epoch 7/150
644/644 [==============================] - 17s 26ms/step - loss: 5.7281 - accuracy: 0.0748 - val_loss: 7.0054 - val_ac

In [14]:
# Saving the Model
model.save("next_word_predicter_lstm.h5")